In [ ]:
from __future__ import unicode_literals

In [ ]:
import pymongo

In [ ]:
import json
import tweepy
from tweepy import API

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [ ]:
import datetime
import re
from bson import json_util
from collections import Counter
import itertools

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from networkx.drawing import draw_spring

%matplotlib inline

In [ ]:
try:
    conn = pymongo.MongoClient()
    print("Connected successfully!!!")
except pymongo.errors.ConnectionFailure, e:
   print("Could not connect to MongoDB: {}".format(e)) 
conn

In [ ]:
db = conn.tweets

In [ ]:
db

In [ ]:
conn.database_names()

In [ ]:
collection = db.tweets

In [ ]:
collection

In [ ]:
db.collection_names()

In [ ]:
collection.insert_one({'name' : 'Alberto', 'surname' : 'Negron', 'twitter' : '@Altons'})

In [ ]:
conn.database_names()

In [ ]:
db.collection_names()

In [ ]:
consumer_key = #Put the consumer key here
consumer_secret= #Put the consumer secret here

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [ ]:
auth_url = auth.get_authorization_url()
print('Please authorize: {}'.format(auth_url))

In [ ]:
TWEET_PIN = #Write your pin here
auth.get_access_token(TWEET_PIN)

In [ ]:
print('ACCESS_KEY: {}'.format(auth.access_token))
print('ACCESS_SECRET: {}'.format(auth.access_token_secret))

In [ ]:
api = tweepy.API(auth)

In [ ]:
lookup = 'terrorism'

for status in tweepy.Cursor(api.search, q=lookup, lang = 'en', count = 100).items(50):
    #creation_date = datetime.datetime.strptime(status.created_at, "%Y-%m-%d %H:%M:%S")
    tweet_result = dict()
    tweet_result['created_at'] = status.created_at 
    tweet_result['from_user'] = dict()
    tweet_result['from_user']['id'] = status.user.id
    tweet_result['from_user']['id_str'] = status.user.id_str
    tweet_result['from_user']['name'] = status.user.name
    tweet_result['from_user']['followers_count'] = status.user.followers_count
    tweet_result['geo'] = status.geo
    tweet_result['id']= status.id
    tweet_result['iso_language_code']= status.lang
    tweet_result['source']= status.source
    tweet_result['text']= status.text
    tweet_result['in_reply_to'] = dict()
    tweet_result['in_reply_to']['user_id'] = status.in_reply_to_user_id
    tweet_result['in_reply_to']['user_id_str'] = status.in_reply_to_user_id_str
    tweet_result['in_reply_to']['user_name'] = status.in_reply_to_screen_name
    db.terrorTweets.insert(tweet_result)

In [ ]:
with open('tweetweet.json', 'wb') as jf:
    json.dump([doc for doc in db.terrorTweets.find({})], jf, indent = 4, default = json_util.default)

In [ ]:
db.terrorTweets.count()

In [ ]:
db.terrorTweets.find({'source' : {'$eq' : 'Twitter for Android'}}).count()

In [ ]:
def printSource():
    for tw in db.terrorTweets.find({}):
        print(tw['source'])

In [ ]:
def countSource():
    sourceCounter = Counter()
    for tw in db.terrorTweets.find({}):
        sourceCounter[tw['source']] += 1
    return sourceCounter

In [ ]:
def countWords():
    wordCount = Counter()
    wordList = list()
    stopwords = ['the', 'this', 'that', 'of', 'a', 'an', 'to', 'and', 'for', 'with', 'we', 'is', 'are', '', 'in', 'on', 'why',
                 'what', 'up', 'be', 'under', 'rt', 'us', 'our', 'it', 'by', 'they', 'at', 'will', 'about', 'you', 'not',
                 'all', 'should', 'have', 'has', 'no', 'if', 'as', '&amp', 'but', 'than', 'out', 'very', 'more']
    for tw in db.terrorTweets.find({}):
        text = tw['text']
        text = re.sub(r'[,.:;?!#@\-]', '', text.strip().lower())
        wordList.append(text.split(' '))
    for wd in itertools.chain.from_iterable(wordList):
        if wd not in stopwords:
            wordCount[wd] +=1
    return wordCount

In [ ]:
wdcnt = countWords()

In [ ]:
def getMostCommons(collection, k =10):
    mostCommons = list()
    for items in collection.most_common(k):
        mostCommons.append(items[0])
    print(mostCommons)
    regex = re.compile(r'^.*{}.*$'.format('|'.join(mostCommons)), re.I|re.S)
    for tweet in db.terrorTweets.find({'text' : regex}):
        print('{} : {}'.format(tweet['from_user']['name'], tweet['text'].strip()))

In [ ]:
getMostCommons(countWords(), 20)

In [ ]:
delRes = collection.delete_many({})

In [ ]:
delRes.deleted_count

In [ ]:
db.terrorTweets.drop()

In [ ]:
db.collection_names()

In [ ]:
conn.database_names()

In [ ]:
mpl.rc('figure', figsize = (18, 10))
plt.bar(np.arange(1, 26, 1), [x[1] for x in countWords().most_common(25)])
plt.title('25 Most Common Terms')
plt.xticks(np.arange(1, 26, 1), [x[0] for x in countWords().most_common(25)], rotation = 90);

In [ ]:
mpl.rc('figure', figsize = (9, 5))
plt.bar(np.arange(1, len(countSource())+1, 1), [x for x in countSource().values()])
plt.title('25 Most Common Terms')
plt.xticks(np.arange(1, len(countSource())+1, 1), [x for x in countSource().keys()], rotation = 90);

In [ ]:
g = nx.DiGraph()